<table>
  <tr>
    <td><div align="left"><font size="30">Finding blobs</font></div></td>
    <td><img src="images/RVSS-logo.png" width="400"></td>
  </tr>
</table>

In [ ]:
%matplotlib notebook

import sys
!{sys.executable} -m pip install machinevision-toolbox-python # used to install machinevision-toolbox into the kernel
!{sys.executable} -m pip install opencv-contrib-python==4.4.0.46 # update opencv to 4.4.0

from machinevisiontoolbox import Image
import matplotlib.pyplot as plt
import numpy as np

from spatialmath import SE3
from spatialmath import base

np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# Find objects in a binary image

We start by loading a binary image

In [ ]:
sharks = Image('shark2.png', grey=True)
sharks.disp();

When we look at this we see two white objects, vaguely shark shaped, against a black background.  But the *objects* are a mental construct of ours, the image is simply 250,000 black and white pixels.  How do group adjacent pixels of the same color to find the objects?

This is a very classical computer vision problem that goes by many names: blob analysis, connectivity analysis, region labelling and many more.
Such objects in a binary object are often called *blobs*.

Using this toolbox we simply write

In [ ]:
blobs = sharks.blobs()

and the result is a feature object that describes *blobs*

The `blobs` object can describe a number of blobs, and in this case there are

In [ ]:
len(blobs)

blobs.  The background is also a blob, a black blob, but here we are ignoring that.

This object can be indexed or sliced just like a list.  Each element has a number of properties such as its centroid (centre of mass)

In [ ]:
blobs[0].centroid

area in pixels

In [ ]:
blobs[0].centroid

and a bounding box

In [ ]:
blobs[0].bbox

where the first row is the u-axis range, and the second row is the v-axis range.  Alternatively we can consider the columns: the first column is the top-left coordinate and the second column is the bottom-right coordinate.

In [ ]:
sharks.disp()
blobs.plot_labelbox(color='yellow', thickness=2)

which depicts and labels each blob.  We could also mark the centroids by

In [ ]:
blobs.plot_centroid()

which add a marker to the plot above.

# Finding objects in the pixel classification image

A key part of your project will be to develop a deep network to classify each pixel in the image according to whether it belongs to the background, an apple, pear, banana etc.  Each pixel will have a unique value that represents the object class.

We will start by loading a typical image

In [ ]:
fruitclass = Image('images/output_9.png', grey=True)
fruitclass.disp();

If we hover the mouse cursor over the image, at the bottom right we see the pixel values.  The black background has values of zero, and there are regions with non-zero values that represent particular fruit.  Note that the estimates are far from perfect, there might appear to be big chunks of a fruit missing.

To find which fruit are present we can compute a histogram

In [ ]:
hist = fruitclass.hist()
hist.plot()

Unlike a grey scale image, there are very few unique grey levels in this image, and they are all small values.  You will need to zoom into the histogram to see this.

**Q: There are 3 fruit present, what are their class labels?**

The `hist` object has a number of attributes.  `h` is a 256x1 matrix of histogram values.  For a color image it would be a 256x3 matrix.  `x` is a 

Let make a binary image that shows just the pixels that have a value of 3

In [ ]:
pear = fruitclass == 3
pear.disp();

and we can see that there three blobs here, but due to imperfect pixel classification the middle part of the right-hand pear is missing.

In [ ]:
blobs = pear.blobs()

In [ ]:
print(blobs)

Once again, we can highlight these blob in the image by

In [ ]:
pear.disp()
blobs.plot_labelbox(color='yellow', thickness=2)

# Morphological cleanup

We will now explore a family of computer vision algorithms referred to as mathematical morphology -- morphology because they are concerned with shape.

We will create a small image of a disk

In [ ]:
se = Image.kcircle(3)
se

which is simply a $7 \times 7$ array with a circular patch of one valued elements.  If we display it as an image

In [ ]:
plt.figure()
plt.imshow(se, cmap='gray');

we see that it is a bit ragged but it is hard to do much better with such a small grid.

Now we will perform a morphological *dilation* operation using this disk

In [ ]:
dilated = pear.dilate(se)
dilated.disp();

We can think of this as moving the disk to every point in the image, and if *any* point on the disk touches a one-valued pixel, the result at that location is a one.  Clearly it has filled in the hole, but it has made the blobs bigger.

Now we can perform the inverse operation called morphological *erosion*

In [ ]:
eroded = dilated.erode(se)
eroded.disp();

We can think of this operation as moving the disk to every point in the image, and if *every* point on the disk touches a one-valued pixel, the result at that location is a one.  Clearly it has reduced the blobs back to something like their original size, and it has filled in the hole.

Choosing the radius of the structuring element requires a bit of trial and error (and experience).  Too small and it won't fill the gap, too big and the shape of the object will be lost.

This pair of operations, dilating *then* eroding, is referred to as *closing*.  The `close` method will do this for you in a single hit.  

The converse operation, eroding *then* dilating, is referred to as *opening*. It is useful for the case where two separate objects have been joined together.  This is performed by the `open` method. 

Now we can perform another blob analysis

In [ ]:
blobs = eroded.blobs()
print(blobs)
blobs.plot_centroid()

## Your turn!

* Find the blobs for other two fruits in this image
* Try this out on one of your own images
* Write the code described below 

In [ ]:
# Q.  Write code to find the coordinate of the middle of the bottom of the bounding box.  Each blob has a number
#     of useful attributes:
#          uc    horizontal coordinate of the centroid
#          vc    vertical coordinate of the centroid
#          umin  leftmost horizontal coordinate of the blob
#          umax  rightmost horizontal coordinate of the blob
#          vmin  topmost vertical coordinate of the blob
#          vmax  bottommost vertical coordinate of the blob
#
# The blobs object is also an iterator.



